In [1]:
from mleap.analyze_results import AnalyseResults
from mleap.data import Data
import pandas as pd
from mleap.estimators.estimators import instantiate_default_estimators

import matplotlib.pyplot as plt
pd.options.display.max_rows = 1000

/media/viktor/Data/PhD/mleap/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
data = Data()
input_io = data.open_hdf5('data/delgado.hdf5', mode='r')
out_io = data.open_hdf5('data/classification.hdf5', mode='a')
analyze = AnalyseResults(hdf5_output_io=out_io, 
                        hdf5_input_io=input_io, 
                        input_h5_original_datasets_group='delgado_datasets/', 
                        output_h5_predictions_group='experiments/predictions/')


### All datasets

In [3]:
error_all_datasets = analyze.calculate_error_all_datasets(metric='mean_squared_error')

#### Simple average

In [4]:
res_df = analyze.calculate_average_std(error_all_datasets)
res_df

,avg,std
BaggingClassifier,9.678659,56.127002
RandomForestClassifier,10.411836,57.872823
LogisticRegression,12.885971,74.313996
GaussianNaiveBayes,13.723310,69.132686
BernoulliNaiveBayes,16.919236,98.644095
Lasso,17.556042,99.268908
LassoLars,17.618406,99.500173
RidgeRegression,17.728869,100.070785
PassiveAggressiveClassifier,20.188100,118.214050
GradientBoostingClassifier,24.048330,145.624963


#### Cohen's d

In [5]:
cohens_d = analyze.cohens_d(error_all_datasets)
cohens_d

,Cohen's d
BaggingClassifier-BaselineClassifier,0.195295
BaselineClassifier-RandomForestClassifier,-0.190966
BaselineClassifier-LogisticRegression,-0.174502
BaselineClassifier-GaussianNaiveBayes,-0.170901
BaggingClassifier-SVC,0.159332
RandomForestClassifier-SVC,0.153453
BaselineClassifier-BernoulliNaiveBayes,-0.148160
BaselineClassifier-Lasso,-0.144690
BaselineClassifier-LassoLars,-0.144317
BaselineClassifier-RidgeRegression,-0.143624


#### t-test

In [6]:
t_test, t_test_df = analyze.t_test(error_all_datasets)
t_test_df

,pair,t_statistic,p_value
0,BaggingClassifier - BaselineClassifier,-1.512750,0.131659
1,BaggingClassifier - BernoulliNaiveBayes,-0.698861,0.485316
2,BaggingClassifier - GaussianNaiveBayes,-0.497562,0.619249
3,BaggingClassifier - GradientBoostingClassifier,-1.008618,0.314174
4,BaggingClassifier - Lasso,-0.756702,0.449971
5,BaggingClassifier - LassoLars,-0.761348,0.447197
6,BaggingClassifier - LogisticRegression,-0.377270,0.706307
7,BaggingClassifier - PassiveAggressiveClassifier,-0.879746,0.379877
8,BaggingClassifier - RandomForestClassifier,-0.099623,0.920727
9,BaggingClassifier - RidgeRegression,-0.768594,0.442890


#### sign test

In [7]:
sign_test, sign_test_df = analyze.sign_test(error_all_datasets)
sign_test_df

,pair,t_statistic,p_value
0,BaggingClassifier - BaselineClassifier,-7.382920,1.548550e-13
1,BaggingClassifier - BernoulliNaiveBayes,-2.830119,4.653064e-03
2,BaggingClassifier - GaussianNaiveBayes,-3.367309,7.590546e-04
3,BaggingClassifier - GradientBoostingClassifier,-0.557392,5.772596e-01
4,BaggingClassifier - Lasso,-1.354454,1.755917e-01
5,BaggingClassifier - LassoLars,-0.562902,5.735018e-01
6,BaggingClassifier - LogisticRegression,-0.867769,3.855210e-01
7,BaggingClassifier - PassiveAggressiveClassifier,-1.528007,1.265107e-01
8,BaggingClassifier - RandomForestClassifier,0.468320,6.395561e-01
9,BaggingClassifier - RidgeRegression,-0.539027,5.898685e-01


#### t-test with Bonferroni correction

In [8]:
t_test_bonferroni, t_test_bonferroni_df = analyze.t_test_with_bonferroni_correction(error_all_datasets)
t_test_bonferroni_df

,pair,p_value
0,BaggingClassifier - BaselineClassifier,1.0
1,BaggingClassifier - BernoulliNaiveBayes,1.0
2,BaggingClassifier - GaussianNaiveBayes,1.0
3,BaggingClassifier - GradientBoostingClassifier,1.0
4,BaggingClassifier - Lasso,1.0
5,BaggingClassifier - LassoLars,1.0
6,BaggingClassifier - LogisticRegression,1.0
7,BaggingClassifier - PassiveAggressiveClassifier,1.0
8,BaggingClassifier - RandomForestClassifier,1.0
9,BaggingClassifier - RidgeRegression,1.0


#### Wilcoxon test

In [9]:
wilcoxon_test, wilcoxon_test_df = analyze.wilcoxon_test(error_all_datasets)
wilcoxon_test_df

,pair,statistic,p_value
0,BaggingClassifier - BaselineClassifier,134.0,5.407341e-20
1,BaggingClassifier - BernoulliNaiveBayes,1258.0,8.733553e-10
2,BaggingClassifier - GaussianNaiveBayes,670.0,3.874445e-14
3,BaggingClassifier - GradientBoostingClassifier,1640.0,4.059521e-05
4,BaggingClassifier - Lasso,3010.0,7.837665e-02
5,BaggingClassifier - LassoLars,3606.0,9.498837e-01
6,BaggingClassifier - LogisticRegression,2665.0,6.151065e-02
7,BaggingClassifier - PassiveAggressiveClassifier,1672.0,2.388333e-05
8,BaggingClassifier - RandomForestClassifier,2025.0,2.059505e-01
9,BaggingClassifier - RidgeRegression,3557.0,7.298601e-01


#### Friedman test

In [10]:
friedman_test, friedman_test_df = analyze.friedman_test(error_all_datasets)
friedman_test_df

,statistic,p_value
0,479.086023,9.212170e-96


In [11]:
nemeniy_test = analyze.nemenyi(error_all_datasets)
pd.DataFrame(nemeniy_test)

,BaggingClassifier,BaselineClassifier,BernoulliNaiveBayes,GaussianNaiveBayes,GradientBoostingClassifier,Lasso,LassoLars,LogisticRegression,PassiveAggressiveClassifier,RandomForestClassifier,RidgeRegression,SVC
BaggingClassifier,-1.000000e+00,1.330535e-08,0.729783,0.382436,9.999995e-01,0.999787,1.000000e+00,0.999978,0.995824,1.000000e+00,1.000000e+00,0.991555
BaselineClassifier,1.330535e-08,-1.000000e+00,0.012870,0.077680,7.213707e-07,0.000012,1.469692e-07,0.000003,0.000101,6.439114e-10,1.168595e-07,0.000187
BernoulliNaiveBayes,7.297835e-01,1.286963e-02,-1.000000,1.000000,9.478722e-01,0.994196,8.836575e-01,0.982193,0.999650,4.863279e-01,8.717527e-01,0.999888
GaussianNaiveBayes,3.824360e-01,7.768015e-02,1.000000,-1.000000,7.354808e-01,0.922646,5.939890e-01,0.854209,0.984081,1.825839e-01,5.729845e-01,0.991474
GradientBoostingClassifier,9.999995e-01,7.213707e-07,0.947872,0.735481,-1.000000e+00,1.000000,1.000000e+00,1.000000,0.999974,9.998811e-01,1.000000e+00,0.999901
Lasso,9.997869e-01,1.211277e-05,0.994196,0.922646,1.000000e+00,-1.000000,9.999963e-01,1.000000,1.000000,9.953856e-01,9.999940e-01,1.000000
LassoLars,1.000000e+00,1.469692e-07,0.883657,0.593989,1.000000e+00,0.999996,-1.000000e+00,1.000000,0.999681,9.999933e-01,1.000000e+00,0.999128
LogisticRegression,9.999778e-01,3.492093e-06,0.982193,0.854209,1.000000e+00,1.000000,9.999999e-01,-1.000000,0.999999,9.989053e-01,9.999998e-01,0.999996
PassiveAggressiveClassifier,9.958239e-01,1.013333e-04,0.999650,0.984081,9.999742e-01,1.000000,9.996813e-01,0.999999,-1.000000,9.668151e-01,9.995708e-01,1.000000
RandomForestClassifier,1.000000e+00,6.439114e-10,0.486328,0.182584,9.998811e-01,0.995386,9.999933e-01,0.998905,0.966815,-1.000000e+00,9.999958e-01,0.946709


### Per dataset

In [12]:
error_per_dataset, error_per_dataset_df = analyze.calculate_error_per_dataset(metric='mean_squared_error')
error_per_dataset_df

score  \
abalone                      BaggingClassifier             0.703766   
                             BaselineClassifier            1.174110   
                             BernoulliNaiveBayes           0.871763   
                             GaussianNaiveBayes            0.782328   
                             GradientBoostingClassifier    0.706337   
                             Lasso                         0.643337   
                             LassoLars                     0.613381   
                             LogisticRegression            0.715011   
                             PassiveAggressiveClassifier   0.756408   
                             RandomForestClassifier        0.706337   
                             RidgeRegression               0.613876   
                             SVC                           0.657417   
acute_inflammation           BaggingClassifier             0.000000   
                             BaselineClassifier            0.651920   
                             BernoulliNaiveBayes           0.353553   
                             GaussianNaiveBayes            0.418330   
                             GradientBoostingClassifier    0.000000   
                             Lasso                         0.339651   
                             LassoLars                     0.166508   
                             LogisticRegression            0.316228   
                             PassiveAggressiveClassifier   0.316228   
                             RandomForestClassifier        0.000000   
                             RidgeRegression               0.167097   
                             SVC                           0.000000   
acute_nephritis              BaggingClassifier             0.000000   
                             BaselineClassifier            0.670820   
                             BernoulliNaiveBayes           0.158114   
                             GaussianNaiveBayes            0.158114   
                             GradientBoostingClassifier    0.000000   
                             Lasso                         0.232971   
                             LassoLars                     0.162721   
                             LogisticRegression            0.000000   
                             PassiveAggressiveClassifier   0.000000   
                             RandomForestClassifier        0.000000   
                             RidgeRegression               0.165912   
                             SVC                           0.000000   
adult                        BaggingClassifier             0.389160   
                             BaselineClassifier            0.600544   
                             BernoulliNaiveBayes           0.442850   
                             GaussianNaiveBayes            0.437281   
                             GradientBoostingClassifier    0.369366   
                             Lasso                         0.397772   
                             LassoLars                     0.354561   
                             LogisticRegression            0.397208   
                             PassiveAggressiveClassifier   0.399233   
                             RandomForestClassifier        0.373043   
                             RidgeRegression               0.354567   
                             SVC                           0.401016   
annealing                    BaggingClassifier             0.301511   
                             BaselineClassifier            0.846860   
                             BernoulliNaiveBayes           0.622258   
                             GaussianNaiveBayes            0.951695   
                             GradientBoostingClassifier    0.317821   
                             Lasso                         0.607800   
                             LassoLars                     0.529761   
                             LogisticRegression            0.565566   
               